<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Разделение-признаков-на-обучающие-и-тестовые" data-toc-modified-id="Разделение-признаков-на-обучающие-и-тестовые-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Разделение признаков на обучающие и тестовые</a></span></li><li><span><a href="#Обучение-модели-без-преобразования" data-toc-modified-id="Обучение-модели-без-преобразования-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Обучение модели без преобразования</a></span></li><li><span><a href="#Преобразование-матрицы-признаков" data-toc-modified-id="Преобразование-матрицы-признаков-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Преобразование матрицы признаков</a></span></li><li><span><a href="#Обучение-модели-после-преобразования" data-toc-modified-id="Обучение-модели-после-преобразования-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Обучение модели после преобразования</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем необходимые нам библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import warnings
warnings.filterwarnings("ignore")

**Загрузим данные и сохраним их в переменной `data`. Выведем информацию о датасете**

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Выведем на экран первые 5 строк

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Проверим датасет на наличие пропусков, а так же посмотрим уникальные знаяения столбца `Страховые выплаты`, который является Целевым признаком

In [4]:
print('Пропусков в датасете: ', data.isna().sum())
print('Уникальные значения Целевого признака: ', data['Страховые выплаты'].unique())

Пропусков в датасете:  Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64
Уникальные значения Целевого признака:  [0 1 2 3 5 4]


In [5]:
data['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

In [6]:
data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Мы видим, что в нашем датасете 5000 значений, пропуски отсутствуют, а целевой признак имеет 6 категорий. Кроме того присутствует разбалансировка классов. Еще мы заметили, что Возраст оказывает на страховые выплаты гораздо большее влияние, чем остальные признаки. Приступим к дальнейшему анализу

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.***

**Задание:
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:** 
Запишем предсказания, полученные после умножения матрицы $Х$ на обратимую матрицу $P$:
$$
a' = XPw'
$$

Тогда формула обучения будет иметь вид:
$$
w'= ((XP)^T XP)^{-1} (XP)^T y
$$
Преобразуем эту формулу:
$w' = ((XP)^T XP)^{-1} (XP)^T y = (P^TX^T XP)^{-1} P^TX^T y = (P^T (X^TX) P)^{-1} P^TX^T y = 
= /Так как X^TX - квадратная матрица, то получим/ = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y = 
= P^{-1} (X^TX)^{-1} E X^T y = P^{-1} (X^TX)^{-1}  X^T y = P^{-1} w$
Мы получили следующее отношение между w и w'
$$
w' = P^{-1} w 
$$
Теперь запишем предсказания, полученные после умножения матрицы $Х$ на обратимую матрицу $P$:
$$a' = XPw' = XPP^{-1} w = X w = a$$

Следовательно при умножении матрицы на обратимую матрицу наши предсказания не изменятся, а значит качество модели останется прежним
 

## Алгоритм преобразования

**Алгоритм**

Для защиты информации будим умножать матрицу признаков на обратимую матрицу Z, сформированную случайным образом. Размерность матрицы будет равна `n*n`, где n - количество признаков, по которым мы будем обучать модель

**Обоснование**

Как было доказано выше, при умножении матрицы признаков на обратимую матрицу, качество можели не меняется.

## Проверка алгоритма

### Разделение признаков на обучающие и тестовые

In [8]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Обучение модели без преобразования

Обучим модель с помощью линейной регрессии и найдем значение метрики R2

In [9]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
R2_data = r2_score(target_test, predictions)
print('R2 модели без преобразования: ', R2_data)

R2 модели без преобразования:  0.43522757127026546


Создадим функцию, которая преобразует нашу матрицу признков, умножив ее на случайную обратимую матрицу


In [10]:
def new_features(features):
    features_new = features
    n = features.shape[1]
    matrix = np.random.random((n,n))
    det_matrix = np.linalg.det(matrix)
    while det_matrix ==0:
        matrix = np.random.random((n,n))
    features_new =  features_new @ matrix
    return features_new, matrix

### Преобразование матрицы признаков

In [11]:
features, matrix = new_features(features)
print(features.head())
print(matrix)

             0             1             2             3
0  5222.905425  38135.116904  23124.866910  19770.651497
1  4004.169098  29229.706796  17717.864796  15158.250725
2  2213.313655  16156.169752   9791.657642   8379.665710
3  4388.883766  32049.240779  19440.651647  16610.643282
4  2749.379112  20072.809481  12169.027094  10408.831524
[[0.03547571 0.40458577 0.08872609 0.59429779]
 [0.18513937 0.92082726 0.08619731 0.80694672]
 [0.10514022 0.7680698  0.46615038 0.39791735]
 [0.32433917 0.69641119 0.18538816 0.27204691]]


### Обучение модели после преобразования 

Опять разобьем данные на обучающую и тестовую и обучим модель

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
R2_data_new = r2_score(target_test, predictions)
print('R2 модели после преобразования: ', R2_data_new)

R2 модели после преобразования:  0.43522757127084777


Сравним показатели метрики R2 до и после преобразования

In [13]:
print('R2 модели без преобразования: ', R2_data)
print('R2 модели после преобразования: ', R2_data_new)

R2 модели без преобразования:  0.43522757127026546
R2 модели после преобразования:  0.43522757127084777


# Наши метрики совпадают с точностью до 10 знаков после запятой, значит мы правильно выполнили шифрование и обезопасили данные пользователей.

# Вывод
# В ходе выполнения проекта были сделаны следующие действия:
- Изучены данные
- Доказано, что при умножении признаки на обратимую матрицу, качество линейной регрессии не изменяется.
- На основания утверждения, описанного выше, создан алгоритм преобразования данных для их защиты
- Осуществлено преобразование данных и изучена метрика R2 для данных до преобразования и после
 
 **В результате метрики R2 для данных до преобразования и после оказались одинаковыми, а значит мы получили зашифрованные данные.**